In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b0078b984692ca1ef687865ec01242b8e91835f287697084ad5fe70f8eb26b91
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
from IPython.core.display import HTML,display
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
# Assuming you've uploaded the CSV to Colab
df = spark.read.csv("/content/PhonePe_Sherloc_Categories_updated_Smaller_One.csv", header=True, inferSchema=True)
df.show()

+--------------------+----------------+------------------------+-------------------------+--------------------+--------------------+------------------+-----------+----------+--------------+------------+--------------------+------------------+-----------+--------------------+--------------------+------------+-------+----------------+----------------+-----------------+--------------------+
|  upi_transaction_id|transaction_type|transaction_reference_id|customer_reference_number|                note|payer_account_number|payer_account_type| payer_ifsc|payer_name|     payer_vpa|payer_amount|payee_account_number|payee_account_type| payee_ifsc|          payee_name|           payee_vpa|payee_amount|  state|         created|         updated|        Category1|           Category2|
+--------------------+----------------+------------------------+-------------------------+--------------------+--------------------+------------------+-----------+----------+--------------+------------+----------------

In [6]:
from pyspark.ml.feature import StringIndexer

# Select the relevant columns
df2 = df.select(['payer_name','payer_vpa','payee_account_type','payee_name','payee_vpa','payer_account_type', 'Category1','Category2'])

# Convert string labels to numeric
label_indexer_cat1 = StringIndexer(inputCol="Category1", outputCol="label_cat1").fit(df2)
label_indexer_cat2 = StringIndexer(inputCol="Category2", outputCol="label_cat2").fit(df2)
df2 = label_indexer_cat1.transform(df2)
df2 = label_indexer_cat2.transform(df2)

# Tokenize the columns
tokenizer_inputs = ['payer_name', 'payee_name', 'payee_account_type', 'payer_account_type', 'payer_vpa', 'payee_vpa']
for inputCol in tokenizer_inputs:
    tokenizer = Tokenizer(inputCol=inputCol, outputCol=inputCol+"_tokens")
    df2 = tokenizer.transform(df2)

# Use HashingTF and IDF to convert tokens to features
for inputCol in tokenizer_inputs:
    hashingTF = HashingTF(inputCol=f"{inputCol}_tokens", outputCol=f"{inputCol}_tf")
    df2 = hashingTF.transform(df2)
    idf = IDF(inputCol=f"{inputCol}_tf", outputCol=f"{inputCol}_features").fit(df2)
    df2 = idf.transform(df2)

# Assemble features
assembler = VectorAssembler(inputCols=[f"{inputCol}_features" for inputCol in tokenizer_inputs], outputCol="features")
df2 = assembler.transform(df2)

# Split data
train, test = df2.randomSplit([0.8, 0.2], seed=42)

# Train Random Forest models
rf_cat1 = RandomForestClassifier(labelCol="label_cat1", featuresCol="features")
rf_cat2 = RandomForestClassifier(labelCol="label_cat2", featuresCol="features")

model_cat1 = rf_cat1.fit(train.select("features", "label_cat1"))
model_cat2 = rf_cat2.fit(train.select("features", "label_cat2"))

# Predictions
predictions_cat1 = model_cat1.transform(test)
predictions_cat2 = model_cat2.transform(test)

# Evaluate
evaluator_cat1 = MulticlassClassificationEvaluator(labelCol="label_cat1", predictionCol="prediction", metricName="accuracy")
accuracy_cat1 = evaluator_cat1.evaluate(predictions_cat1)
print("Accuracy for Category Level 1: %.2f" % accuracy_cat1)

evaluator_cat2 = MulticlassClassificationEvaluator(labelCol="label_cat2", predictionCol="prediction", metricName="accuracy")
accuracy_cat2 = evaluator_cat2.evaluate(predictions_cat2)
print("Accuracy for Category Level 2: %.2f" % accuracy_cat2)



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored